In [2]:
!pip install azure-storage-file

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 47 kB 2.8 MB/s 
     |████████████████████████████████| 4.1 MB 10.8 MB/s 


In [4]:
# file Share와 연결하는 코드
from azure.storage.file import FileService
file_service = FileService(account_name='',
                           account_key='')

In [5]:
# myshare라는 공유 폴더를 생성
file_service.create_share('myshare')

True

In [6]:
# myshare 안에 샘플 디렉토리인 sampledir을 생성
file_service.create_directory('myshare','sampledir')

True

In [8]:
# 파일 업로드(ml_map.jpg를 myfile로 해당 공유 폴더에 복사)
from azure.storage.file import ContentSettings

file_service.create_file_from_path('myshare',
                                   None,  #하위폴더를 지정안할거라 None으로 표시
                                   'myfile',
                                   'ml_map.jpg', # ml_map.jpg를 myfile의 이름으로 업로드
                                   content_settings=ContentSettings(content_type='image/jpg'))

In [9]:
공유 폴더안에 파일 목록을 가져와서 보여준다.
generator = file_service.list_directories_and_files('myshare')
for file_or_dir in generator:
  print(file_or_dir.name)

myfile
sampledir


In [10]:
# 다시 myfile로 복사된 ml_map.jpg를 out-ml_map.jpg로 가져온다. 
file_service.get_file_to_path('myshare',None,'myfile','out-ml_map.jpg') # myfile을 out-ml_map.jpg로 다운로드

In [11]:
# 코드로 스냅샷 추가하기(스냅샷을 찍어둔다.)
metadata = {"foo": "bar"}
snapshot = file_service.snapshot_share('myshare', metadata=metadata) # 스냅샷이 추가됨. 내가 원하는 공유 폴더들의 목록을 뽑고 할 때 메타데이터로 찾게 됨.

In [16]:
# 전체 스냅샷의 목록을 가져온다.
shares = list(file_service.list_shares(include_snapshots=True))

for s in shares:
  print(s.snapshot)

2022-11-08T00:22:42.0000000Z
2022-11-08T00:24:23.0000000Z
None


In [13]:
# 스냅샷의 파일과 디렉토리 목록을 가져온다. 
directories_and_files = list(
    file_service.list_directories_and_files('myshare', snapshot='2022-11-08T00:24:23.0000000Z')) # 방금 위에서 만든 스냅샷의 이름

In [14]:
# 파일과 디렉토리 목록을 출력한다. 
directories_and_files
for file_or_dir in generator:
  print(file_or_dir.name)

myfile
sampledir


In [17]:
# 스냅샷의 삭제 

file_service.delete_share('myshare', snapshot='2022-11-08T00:24:23.0000000Z')

True

In [18]:
# 공유 폴더 내의 파일 삭제
file_service.delete_file('myshare', None, 'myfile')

In [20]:
# 공유 폴더의 삭제
file_service.delete_share('myshare') 
# 처음에는 에러가 남. 이유는 스냅샷이 남아있는 상태에서는 폴더 삭제가 안됨. 안전장치 역할. 
# 처음 portal에서 수동으로 만든 파일이 남아있어서 삭제가 안된 것. 그러므로 수동 삭제 후 코드 실행

True